In [1]:
# The Snowpark package is required for Python Worksheets. 
# You can add more packages by selecting them using the Packages control and then importing them.

import snowflake.snowpark as snowpark
import requests
from snowflake.snowpark.functions import col
import pandas as pd

def get_linkedin_data_engineer_mgr_daily():
    api_key = 'aiP2dQo2OXOEWUCeCSXiqA'
    headers = {'Authorization': 'Bearer ' + api_key}
    api_endpoint = 'https://nubela.co/proxycurl/api/v2/linkedin/company/job'
    #  api_endpoint = 'https://google.com'
    params = {
        'job_type': 'anything',
        'experience_level': 'mid_level',
        'when': 'past-day',
        'flexibility': 'remote',
        'geo_id': '92000000',
        'keyword': 'data engineering manager',
    }
    response = requests.get(api_endpoint,
                            params=params,
                            headers=headers)

    max_rows=100
    rows=0
    j = response.json()
    if j and 'job' in j:
        df = pd.DataFrame.from_records(j['job'])

        rows += len(j['job'])
        while(j['next_page_api_url'] and rows<max_rows):
            response = requests.get(j['next_page_api_url'],
                                    params=params,
                                    headers=headers)
            j = response.json()
            if j and 'job' in j:
                dfx = pd.DataFrame.from_records(j['job'])
                df = pd.concat([df, dfx], ignore_index=True)
                rows += len(j['job'])
    
    return df
    
def main(session: snowpark.Session): 
    # Your code goes here, inside the "main" handler.

    df = get_linkedin_data_engineer_mgr_daily()

    # Return value will appear in the Results tab.
    return df

ModuleNotFoundError: No module named 'snowflake'

In [50]:
from snowflake.snowpark import Session
connection_parameters = {
    "user":"louspringer",
    "account":'CFCLGQP-HQB93436',
    "private_key_file":'/home/lou/snow-keys/rsa_key.p8',
    "warehouse":"PRD_ALV_WAREHOUSE",
    "database":"PRD_ALV",        
    "schema":'QUERY_LINKEDIN',
    "role":'ALV_PROD_ROLE',
}
session = Session.builder.configs(connection_parameters).create()

In [38]:
df = main(session)
df

,company,company_url,job_title,job_url,list_date,location
0,Meta,https://www.linkedin.com/company/meta,"Data Engineering Manager, Analytics",https://www.linkedin.com/jobs/view/data-engine...,2024-03-05,Stati Uniti d'America
1,Green Dot Corporation,https://www.linkedin.com/company/green-dot-cor...,Remote - Sr. Data Engineering Manager,https://www.linkedin.com/jobs/view/remote-sr-d...,2024-04-25,Stati Uniti d'America
2,Airbnb,https://www.linkedin.com/company/airbnb,"Engineering Manager, Data Foundation",https://www.linkedin.com/jobs/view/engineering...,2024-03-27,"San Francisco, California, Stati Uniti d'America"
3,"Solvd, Inc.",https://www.linkedin.com/company/solvdinc,Data Engineering Manager,https://www.linkedin.com/jobs/view/data-engine...,2024-04-25,Romania
4,Xcede,https://www.linkedin.com/company/xcede,Data Engineering Manager,https://www.linkedin.com/jobs/view/data-engine...,2024-05-02,Regno Unito
...,...,...,...,...,...,...
95,Healthcare.com,https://www.linkedin.com/company/healthcare-com,Software Engineering Manager,https://www.linkedin.com/jobs/view/software-en...,2024-04-18,Stati Uniti d'America
96,Panasonic North America,https://www.linkedin.com/company/panasonic-nor...,Cloud Engineering Manager,https://www.linkedin.com/jobs/view/cloud-engin...,2024-03-25,Stati Uniti d'America
97,Allergan Data Labs,https://www.linkedin.com/company/allergan-data...,Software Engineering Manager,https://www.linkedin.com/jobs/view/software-en...,2024-02-02,"Irvine, California, Stati Uniti d'America"
98,TransUnion,https://www.linkedin.com/company/transunion,Software Engineering Managers,https://www.linkedin.com/jobs/view/software-en...,2024-03-18,Regno Unito


In [53]:
df.columns = df.columns.str.upper()
df.columns

Index(['COMPANY', 'COMPANY_URL', 'JOB_TITLE', 'JOB_URL', 'LIST_DATE',
       'LOCATION'],
      dtype='object')

In [40]:
from snowflake.connector.pandas_tools import write_pandas
#df.to_sql('jobs01', session)

In [54]:
# Create the connection to the Snowflake database.
cnx = session.connection

# Write the data from the DataFrame to the table named "customers".
success, nchunks, nrows, _ = write_pandas(cnx, df, 'JOB_LISTINGS')

In [32]:
j

{'job': [{'company': 'Meta',
   'company_url': 'https://www.linkedin.com/company/meta',
   'job_title': 'Data Engineering Manager, Analytics',
   'job_url': 'https://www.linkedin.com/jobs/view/data-engineering-manager-analytics-at-meta-3828885964',
   'list_date': '2024-03-05',
   'location': "Stati Uniti d'America"},
  {'company': 'Green Dot Corporation',
   'company_url': 'https://www.linkedin.com/company/green-dot-corporation',
   'job_title': 'Remote - Sr. Data Engineering Manager',
   'job_url': 'https://www.linkedin.com/jobs/view/remote-sr-data-engineering-manager-at-green-dot-corporation-3906304720',
   'list_date': '2024-04-25',
   'location': "Stati Uniti d'America"},
  {'company': 'Airbnb',
   'company_url': 'https://www.linkedin.com/company/airbnb',
   'job_title': 'Engineering Manager, Data Foundation',
   'job_url': 'https://www.linkedin.com/jobs/view/engineering-manager-data-foundation-at-airbnb-3869749922',
   'list_date': '2024-03-27',
   'location': "San Francisco, Cal

In [18]:
df.iloc[0]['job_url']

'https://www.linkedin.com/jobs/view/data-engineering-manager-analytics-at-meta-3828885964'

In [19]:
api_key = 'aiP2dQo2OXOEWUCeCSXiqA'
headers = {'Authorization': 'Bearer ' + api_key}
api_endpoint = 'https://nubela.co/proxycurl/api/v2/linkedin/company/job'
#  api_endpoint = 'https://google.com'
params = {
    'job_type': 'anything',
    'experience_level': 'mid_level',
    'when': 'past-day',
    'flexibility': 'remote',
    'geo_id': '92000000',
    'keyword': 'data engineering manager',
}
response = requests.get(api_endpoint,
                        params=params,
                        headers=headers)

j = response.json()

In [20]:
j

{'job': [{'company': 'Meta',
   'company_url': 'https://www.linkedin.com/company/meta',
   'job_title': 'Data Engineering Manager, Analytics',
   'job_url': 'https://www.linkedin.com/jobs/view/data-engineering-manager-analytics-at-meta-3828885964',
   'list_date': '2024-03-05',
   'location': "Stati Uniti d'America"},
  {'company': 'Green Dot Corporation',
   'company_url': 'https://www.linkedin.com/company/green-dot-corporation',
   'job_title': 'Remote - Sr. Data Engineering Manager',
   'job_url': 'https://www.linkedin.com/jobs/view/remote-sr-data-engineering-manager-at-green-dot-corporation-3906304720',
   'list_date': '2024-04-25',
   'location': "Stati Uniti d'America"},
  {'company': 'Airbnb',
   'company_url': 'https://www.linkedin.com/company/airbnb',
   'job_title': 'Engineering Manager, Data Foundation',
   'job_url': 'https://www.linkedin.com/jobs/view/engineering-manager-data-foundation-at-airbnb-3869749922',
   'list_date': '2024-03-27',
   'location': "San Francisco, Cal

In [22]:
response = requests.get(j['next_page_api_url'],
                        headers=headers)

In [21]:
j['next_page_api_url']

'https://nubela.co/proxycurl/api/v2/linkedin/company/job?pagination=eyJwYWdlIjogMSwgIm1ldGhvZCI6ICJtYnJvd3NlciIsICJqb2JzX29mZnNldCI6IDEwfQ&keyword=data+engineering+manager&geo_id=92000000&flexibility=remote&when=past-day'

In [23]:
response.json()

{'job': [{'company': 'Centrica',
   'company_url': 'https://www.linkedin.com/company/centrica',
   'job_title': 'Data Engineering Manager',
   'job_url': 'https://www.linkedin.com/jobs/view/data-engineering-manager-at-centrica-3913946928',
   'list_date': '2024-04-30',
   'location': 'United Kingdom'},
  {'company': 'Ardent Blue Technologies',
   'company_url': 'https://www.linkedin.com/company/ardent-blue-technologies',
   'job_title': 'Engineering Manager-Python Remote',
   'job_url': 'https://www.linkedin.com/jobs/view/engineering-manager-python-remote-at-ardent-blue-technologies-3920371524',
   'list_date': '2024-05-09',
   'location': 'United States'},
  {'company': 'OneSignal',
   'company_url': 'https://www.linkedin.com/company/onesignal',
   'job_title': 'Engineering Manager',
   'job_url': 'https://www.linkedin.com/jobs/view/engineering-manager-at-onesignal-3905378147',
   'list_date': '2024-04-19',
   'location': 'United States'},
  {'company': 'Attentive',
   'company_url': 